# Version information

In [1]:
# RUN THIS FROM 'py39a' CONDA ENVIRONMENT!
%matplotlib notebook
from PySide2.QtWidgets import *
from datetime import date
print("Running date: ", date.today().strftime("%B %d, %Y"))
# Add cloned repo to system path to avoid having to pip-install as well
import sys
sys.path.append("C:/Users/nmb48/Documents/GitHub/pyleecan")  # Adjust path as needed
import pyleecan
print("Pyleecan version: " + pyleecan.__version__)
import SciDataTool
print("SciDataTool version: " + SciDataTool.__version__)

Running date:  March 19, 2025
Pyleecan version: 1.5.2
SciDataTool version: 2.5.0


## Start the GUI

In [2]:
# %run -m pyleecan

## Import the .json file

In [3]:
import json

with open("conventional_propulsive_motor.json", "r") as f:
    machine_dict = json.load(f)

## Recursively populate the MachineSIPMSM object

In [4]:
%matplotlib inline
from pyleecan.Classes.LamSlotWind import LamSlotWind
from pyleecan.Classes.SlotW21 import SlotW21
from pyleecan.Classes.Winding import Winding
from pyleecan.Classes.CondType12 import CondType12
from pyleecan.Classes.LamSlotMag import LamSlotMag
from pyleecan.Classes.SlotM18 import SlotM18
from pyleecan.Classes.Magnet import Magnet
from pyleecan.Classes.MachineSIPMSM import MachineSIPMSM

# Define a recursive function to assign values from a dictionary to an object
def assign_attributes(obj, data):
    """
    Recursively assigns dictionary values to the attributes of an object.
    
    Parameters:
    obj : The class instance to which attributes should be assigned.
    data : A dictionary containing the attributes and values.
    """
    for key, value in data.items():
        if isinstance(value, dict):  # If value is another dictionary, recurse
            sub_obj = getattr(obj, key, None)
            if sub_obj is not None:  # If sub-object exists, update it
                assign_attributes(sub_obj, value)
            else:  # If sub-object does not exist, assign the dictionary directly
                try:
                    setattr(obj, key, value)
                except Exception as e:
                    print(str(e))
                    
        else:
            try:
                setattr(obj, key, value)  # Assign primitive values
            except Exception as e:
                print(str(e))

# Create main MachineSIPMSM object
conventional_propulsive_motor = MachineSIPMSM(
    stator=LamSlotWind(
        slot=SlotW21(),
        winding = Winding(
            conductor = CondType12()
        ),
    ),
    rotor=LamSlotMag(
        slot = SlotM18(),
        magnet = Magnet()
    ),
    shaft=None,
    frame=None
)

# Recursively assign JSON data
assign_attributes(conventional_propulsive_motor, machine_dict)

# Print to verify
# print(detailed_demonstrator_ch5_populated.stator.Rint)  # Should match JSON value
# print(detailed_demonstrator_ch5_populated.stator.slot.Zs)  # Should be 60

LamSlotWind class has no "R_hex_in" property
LamSlotWind class has no "a" property
LamSlotWind class has no "d" property
MachineSIPMSM class has no "tms" property
MachineSIPMSM class has no "operation" property


### Save .json file and plot cross-section

In [5]:
conventional_propulsive_motor.save('conventional_propulsive_motor_populated.json')

fig, ax = conventional_propulsive_motor.plot(is_show_fig=True)

[10:49:51] Saving MachineSIPMSM to file 'conventional_propulsive_motor_populated.json'.
Saving MachineSIPMSM to file 'conventional_propulsive_motor_populated.json'.


### Plot stator

In [6]:
fig, ax = conventional_propulsive_motor.stator.slot.plot_schematics(is_default=True, is_show_fig=False)
fig, ax = conventional_propulsive_motor.stator.slot.plot(is_show_fig=False)

### Plot winding

In [7]:
fig, ax = conventional_propulsive_motor.stator.plot_winding(is_show_fig=False)   # Show winding pattern smallest sym
fig, ax = conventional_propulsive_motor.stator.winding.plot_linear(is_show_fig=False)
fig, ax = conventional_propulsive_motor.stator.plot(
                is_winding_connection=True,
                is_show_fig=True,
                is_add_sign=True,
            )

AttributeError: 'NoneType' object has no attribute 'text'

### Plot conductor

In [8]:
fig, ax = conventional_propulsive_motor.stator.winding.conductor.plot_schematics(is_default=True, is_single=True, is_show_fig=False)
fig, ax = conventional_propulsive_motor.stator.winding.conductor.plot_schematics(is_default=True, is_single=False, is_show_fig=False)

### Plot magnets

In [9]:
fig, ax = conventional_propulsive_motor.rotor.slot.plot_schematics(is_default=True, is_show_fig=False)
fig, ax = conventional_propulsive_motor.rotor.slot.plot(is_show_fig=False)

In [10]:
from os.path import join
from pyleecan.Classes.Material import Material
from pyleecan.Classes.MatMagnetics import MatMagnetics
from pyleecan.Classes.MatHT import MatHT
from pyleecan.definitions import DATA_DIR
from pyleecan.Functions.load import load

# Loading Materials 
M400_50A = load(join(DATA_DIR, "Material", "M400-50A.json"))
Copper1 = load(join(DATA_DIR, "Material", "Copper1.json"))
Magnet_prius = load(join("C:/Users/nmb48/AppData/Roaming/pyleecan/Material/MagnetPrius.json"))
INSULATOR1 = load(join("C:/Users/nmb48/AppData/Roaming/pyleecan/Material/INSULATOR1.json"))

# Set Materials
conventional_propulsive_motor.stator.mat_type = M400_50A  # Stator Lamination material
conventional_propulsive_motor.rotor.mat_type = M400_50A  # Rotor Lamination material
conventional_propulsive_motor.rotor.magnet.mat_type = Magnet_prius
conventional_propulsive_motor.rotor.magnet.type_magnetization = 0
conventional_propulsive_motor.stator.winding.conductor.ins_mat = INSULATOR1
conventional_propulsive_motor.stator.winding.conductor.cond_mat = Copper1  # Stator winding conductor material

In [11]:
from pyleecan.Classes.MachineSIPMSM import MachineSIPMSM

conventional_propulsive_motor_populated = MachineSIPMSM(
    name="Detailed demonstrator design from chapter 5 of Aidan's thesis",
    stator=conventional_propulsive_motor.stator,
    rotor=conventional_propulsive_motor.rotor,
    shaft=None,
    frame=None
)
conventional_propulsive_motor_populated.save('conventional_propulsive_motor_populated.json')

fig, ax = conventional_propulsive_motor_populated.plot(is_show_fig=False)

[10:50:01] Saving MachineSIPMSM to file 'conventional_propulsive_motor_populated.json'.
Saving MachineSIPMSM to file 'conventional_propulsive_motor_populated.json'.


## Manually edit .json file to add Hmag

Comparing https://www.pyleecan.org/_modules/pyleecan/Classes/SlotM18.html#SlotM18 and https://github.com/NilsBarner/pyleecan/blob/master/pyleecan/Classes/SlotM18.py, it became clear that Hmag was renamed to H0 at some point but the documentation did no get updated. When loading the .json file of the present machine into the GUI and plotting the machine cross-section, the GUI would only recognise Hmag but not H0 https://www.pyleecan.org/pyleecan.GUI.Dialog.DMachineSetup.SMSlot.PMSlot18.PMSlot18.html?highlight=slotm18, hence I had to manually add this to the json.

In [12]:
import json

# Load the JSON file
with open("conventional_propulsive_motor_populated.json", "r") as f:
    data = json.load(f)

data["rotor"]["slot"]["Hmag"] = conventional_propulsive_motor.rotor.slot.H0

# Save the updated JSON
with open("conventional_propulsive_motor_populated.json", "w") as f:
    json.dump(data, f, indent=2)

Upon comparison between the json file created by this Jupyter Notebook and that created by inserting the same values into the GUI in a tool like diffchecker, I confirm that I successfully scripted the detailed machine demonstrator design from chapter 5 of Aidan's thesis.